# Outline of Uber Pickups project

#### 1. Characterize a business need
#### 2. Install useful librairies, retrieve data and process it accordingly to our objective
#### 3. Implement a solution with DBScan
#### 4. Try another solution with K-Means
#### 5. Discussion

## 1. Business need

#### The objective is to create algorithms for Uber that will determine where are the hot-zones that drivers should be in. At least two requirements are identified to achieve this goal:
- create an algorithm to fin hot zones
- visualize results on a dashboard

#### We have at our disposal exhaustive data from Uber for the months of April, May, June, July, August and September 2014. The data contains every Uber ride which took place in New York City and provide the following information :
- data and hour of the ride
- latitude and longitude
- corresponding taxi zone

#### We chose here to focus on working hours (8 am - 20 pm from Monday to Friday) for the Q2-2014 but the code can be easily adapted to any other time period.

## 2. Retrieve and process data

### 2.1 Retrieve data

In [1]:
!pip install plotly -q

In [27]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score

In [3]:
apr = pd.read_csv('data/uber-raw-data-apr14.csv').drop('Base',axis=1)
may = pd.read_csv('data/uber-raw-data-may14.csv').drop('Base',axis=1)
jun = pd.read_csv('data/uber-raw-data-jun14.csv').drop('Base',axis=1)

In [4]:
data = pd.concat([apr, may, jun]).reset_index(drop=True)
print('For the month of Q2-2014, we have {:,} rides.'.format(data.shape[0]))
display(data.head())
display(data.tail())

For the month of Q2-2014, we have 1,880,795 rides.


,Date/Time,Lat,Lon
0,4/1/2014 0:11:00,40.7690,-73.9549
1,4/1/2014 0:17:00,40.7267,-74.0345
2,4/1/2014 0:21:00,40.7316,-73.9873
3,4/1/2014 0:28:00,40.7588,-73.9776
4,4/1/2014 0:33:00,40.7594,-73.9722


,Date/Time,Lat,Lon
1880790,6/30/2014 22:40:00,40.7332,-73.9872
1880791,6/30/2014 23:12:00,40.7905,-73.9796
1880792,6/30/2014 23:13:00,40.7640,-73.9887
1880793,6/30/2014 23:15:00,40.7262,-73.9944
1880794,6/30/2014 23:35:00,40.7404,-73.9848


### 2.2 Process data

#### We process the data in order to keep only working hours (8am-20pm from Monday to Friday).

In [5]:
# We process the data in order to keep only working hours (8am-20pm from Monday to Friday)
data['date'] = pd.to_datetime(data['Date/Time'])
data = data.drop('Date/Time',axis=1)
data['hour'] = data['date'].apply(lambda x : x.hour)
data['week_day'] = data['date'].apply(lambda x : x.weekday()) # the day of the week with Monday=0, Sunday=6.
data['month'] = data['date'].apply(lambda x : x.month)
data = data.drop('date',axis=1)

In [6]:
data.head()

,Lat,Lon,hour,week_day,month
0,40.7690,-73.9549,0,1,4
1,40.7267,-74.0345,0,1,4
2,40.7316,-73.9873,0,1,4
3,40.7588,-73.9776,0,1,4
4,40.7594,-73.9722,0,1,4


## 3. Implement a solution with DBScan

#### DBScan seems adapted to our business need since it is based on the calculation of density which can be interpreted here as the number of rides observed in a given range. By selecting the highest density clusters, we can identify where the 'hot spots' are and recommend to Uber drivers to reach these areas.

#### The quality of the recommendation is directly linked to the density of the clusters which can be determined as follows thanks to the 'epsilon' and 'min_sample' parameters of DBScan:
- low epsilon and high min_sample : high denstity
- high epsilon and low min_sample : low denstity
- low epsilon and low min_sample : high sensitivity to outliers
- high epsilon and high min_sample : low sensitivity to outliers

#### For this project, we decide to identify clusters with at least 100 rides that are located only within a 200 meters distance with each other. Various values have been tried, this combinaison appears to be relevant regarding the business objectives (a Uber user should wait more than 5-7 minutes) and the spatial repartition of the clusters identified.

#### Another advantage of DBScan is that many metrics have been implemented into the sk-learn model. Since the rides are directly identified with GPS coordinates, we will use the haversine distance which computes distance between two points at the surface of the globe based on earth radius and GPS coordinates (expressed in radians, so we will need to copnvert the original data which is in degrees).

In [7]:
earth_radius_km = (6378.137 + 6356.752) / 2 # average earth radius between equatorial and polar radius

In [8]:
# Definition of parameters : epsilon and min_samples
epsilon = 0.2 / earth_radius_km
min_samples = 100

In [9]:
# Creation of variables that will contain the relevant information for the clusters.
results = pd.DataFrame(columns = ['Lat','Lon','week_day','hour','cluster'])
center_points_dict = {'week_day':[], 'hour':[], 'cluster':[], 'lat':[], 'lon':[], 'rides':[]}

In [10]:
%%time
for week_day in range(5):
    for hour in range(8,21):
        print('traitement du jour {:d} et de de l\'heure {:d}.'.format(week_day, hour))
        
        # Selection of relevant data        
        data_temp = data.loc[(data['hour'] == hour) & (data['week_day'] == week_day)].drop(['hour', 'week_day','month'], axis=1)
        coords = np.radians(data_temp) # it's necessary to convert the GPS coordinates into radians
        
        # Fitting of the DBScan
        db = DBSCAN(eps=epsilon, min_samples=100, metric='haversine', algorithm='auto') # use of haversine distance
        db.fit(coords)
        
        # Record of the results : we retrieve the clusters identified thanks to 'db.labels_'
        coords[['Lat','Lon']] = np.degrees(coords[['Lat','Lon']])
        coords['cluster'] = db.labels_
        coords['week_day'] = week_day
        coords['hour'] = hour
        results = pd.concat([results,coords])
        
        # Calculation of the center of each cluster, we first remove the outliers (or noise) rides identified with '-1' in db.labels_
        cluster_list = list(coords['cluster'].unique())
        cluster_list.remove(-1)
        for i in cluster_list:
            data_temp_center = coords.loc[coords['cluster'] == i].drop(['cluster','week_day','hour'],axis=1)
            center_points_dict['cluster'].append(i)
            center_points_dict['rides'].append(data_temp_center.shape[0])
            center_points_dict['lat'].append(np.mean(data_temp_center.Lat))
            center_points_dict['lon'].append(np.mean(data_temp_center.Lon))
            center_points_dict['hour'].append(hour)
            center_points_dict['week_day'].append(week_day)

traitement du jour 0 et de de l'heure 8.
traitement du jour 0 et de de l'heure 9.
traitement du jour 0 et de de l'heure 10.
traitement du jour 0 et de de l'heure 11.
traitement du jour 0 et de de l'heure 12.
traitement du jour 0 et de de l'heure 13.
traitement du jour 0 et de de l'heure 14.
traitement du jour 0 et de de l'heure 15.
traitement du jour 0 et de de l'heure 16.
traitement du jour 0 et de de l'heure 17.
traitement du jour 0 et de de l'heure 18.
traitement du jour 0 et de de l'heure 19.
traitement du jour 0 et de de l'heure 20.
traitement du jour 1 et de de l'heure 8.
traitement du jour 1 et de de l'heure 9.
traitement du jour 1 et de de l'heure 10.
traitement du jour 1 et de de l'heure 11.
traitement du jour 1 et de de l'heure 12.
traitement du jour 1 et de de l'heure 13.
traitement du jour 1 et de de l'heure 14.
traitement du jour 1 et de de l'heure 15.
traitement du jour 1 et de de l'heure 16.
traitement du jour 1 et de de l'heure 17.
traitement du jour 1 et de de l'heure 

In [11]:
# In order to plot the results in a time-animated map, we need to add a proper defined 'time' variable to our 2 dataframes containing the clusters and the centers
# of the clusters.
days_correspondance = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday'}
results['time'] = results['week_day'].replace(days_correspondance) + ', hour : ' + results['hour'].astype(str)
center_points_pd = pd.DataFrame(center_points_dict)
center_points_pd['time'] = center_points_pd['week_day'].replace(days_correspondance) + ', hour : ' + center_points_pd['hour'].astype(str)

#### To show the results, we plot the clusters identified on average for every working hour during Q4.

In [12]:
fig_clusters = px.scatter_mapbox(
        results[results.cluster != -1], 
        lat="Lat", 
        lon="Lon",
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame = 'time'
)

fig_clusters.show("iframe")

#### To bring more accurate business information, we can plot the center of the clusters identified. After another processing of the restuls, the size of the marker is determined by the relative number of rides contained in the cluster compared to the other cluster : the bigger marker, the 'hotter' the spot for a Uber rider.

In [13]:
rides_per_hour = center_points_pd.groupby('time')['rides'].sum().reset_index().rename(columns={'rides':'total_rides_per_hour'})
rides_per_hour

,time,total_rides_per_hour
0,"Friday, hour : 10",1135
1,"Friday, hour : 11",2042
2,"Friday, hour : 12",3064
3,"Friday, hour : 13",5252
4,"Friday, hour : 14",8297
...,...,...
60,"Wednesday, hour : 18",16713
61,"Wednesday, hour : 19",13668
62,"Wednesday, hour : 20",14378
63,"Wednesday, hour : 8",3534


In [14]:
center_points = pd.merge(center_points_pd, rides_per_hour, on='time', how='left')
center_points['rides_indicator'] = center_points['rides'] / center_points['total_rides_per_hour']

In [22]:
fig_centers = px.scatter_mapbox(
        center_points, 
        lat='lat', 
        lon='lon',
        color='cluster',
        size = 'rides_indicator',
        mapbox_style='carto-positron',
        animation_frame = 'time'
)

fig_centers.show("iframe")

## 4. K-Means

#### The purpose of this section is to try to find clusters thanks to K-Means for a specific hour. The results with DBScan will be compared and discussed in the last section.

#### The first step for K-Means is to determined the most suitable number of clusters ('K') thanks to :
- the 'elbow' method : computation of the inertia for various values of K
- the computation of the 'silhouette' for each K

#### We use the first hour identified in our dataset : hour 8 for Monday during Q4.

In [83]:
sample_data = data.loc[(data['hour'] == 8) & (data['week_day'] == 0)].drop(['hour', 'week_day','month'], axis=1)

In [56]:
# We collect the inertia (sum of of squared distances of each observation to its closest cluster center) and the mean silhouette score for each value of K.
# We start at k=2 because this score cannot by definition be computed for less than 2 clusters
# (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

all_k = [i for i in range(2,11)]
inertia = []
silhouette = []
for k in all_k: 
    kmeans = KMeans(n_clusters= k, init = "k-means++", random_state = 0)
    kmeans.fit(sample_data)
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(sample_data, kmeans.predict(sample_data)))
    print("\n**** results for K = {:d}. ****".format(k))
    print("Inertia : {:.1f}".format(inertia[-1]))
    print("Silhouette score : {:.4f}".format(silhouette[-1]))


**** results for K = 2. ****
Inertia : 30.0
Silhouette score : 0.6593

**** results for K = 3. ****
Inertia : 21.8
Silhouette score : 0.4136

**** results for K = 4. ****
Inertia : 15.8
Silhouette score : 0.4464

**** results for K = 5. ****
Inertia : 12.2
Silhouette score : 0.4590

**** results for K = 6. ****
Inertia : 9.3
Silhouette score : 0.4868

**** results for K = 7. ****
Inertia : 8.3
Silhouette score : 0.4922

**** results for K = 8. ****
Inertia : 7.3
Silhouette score : 0.4073

**** results for K = 9. ****
Inertia : 6.4
Silhouette score : 0.4167

**** results for K = 10. ****
Inertia : 5.7
Silhouette score : 0.4334


In [57]:
fig = px.line(pd.DataFrame([inertia, all_k]).T.rename(columns={0:'inertia', 1:'K'}),
              x="K",
              y="inertia",
              title='Elbow method : inertia per number of clusters (K)')
fig.show("iframe")

In [58]:
fig = px.line(pd.DataFrame([silhouette, all_k]).T.rename(columns={0:'silhouette', 1:'K'}),
              x="K",
              y="silhouette",
              title='silhouette per number of clusters (K)')
fig.show("iframe")

#### Considering the shape of the inertia curve, we decide to use K=6 (the steepest change of slope). K=6 seems also consistent regarding the silhouette score as the value is almost the maximum, if we exclude K=2 which doesn't seem to be a good candidate regarding inertia.

In [85]:
kmeans = KMeans(n_clusters= 6, init = "k-means++", random_state = 0)
kmeans.fit(sample_data)
sample_data['cluster'] = kmeans.fit_predict(sample_data)
sample_data['type'] = 'K-Means'

## 5. Discussion

#### Let's compare the results obtained with DBScan and K-means (K=6) for Monday, hour 8 during Q4.

In [111]:
dbscan_results = results.loc[(results['hour'] == 8) & (results['week_day'] == 0) & (results['cluster'] != -1)].drop(['hour', 'week_day','time'], axis=1)
dbscan_results['type'] = 'DBSCAN'

In [115]:
discussion_df = pd.concat([sample_data, dbscan_results]).reset_index(drop=True)

In [117]:
fig_clusters = px.scatter_mapbox(
        discussion_df, 
        lat="Lat", 
        lon="Lon",
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame = 'type'
)

fig_clusters.show("iframe")

#### As a conclusion, we would recommend to use DBSCAN over K-Means for this business case beacuse of the following reasons :
- K-means is more sensitive to outliers, which is an important issue here : we certainly do not want to recommand to a driver to go into a cluster with a density that would be actually too low. On the contrary, DBSCAN is non-sensitive to outliers (they are actually identified as such).
- K-means need to determine the adequate number of clusters for each dataset. It would be complicated to generate the methods used above to determine K=6. On the contrary, DBSCAN determines itself the number of clusters according to the density parameters we chose.£
- The most tricky part with DBSCAN is the tuning of epsilon and min_samples parameters but it makes also it very suitable for our business need : we are able to exploit this algorithm with very concrete parameters : max distance between rides to be picked-up and minimum number of rides within an area. As a perspective, we could consider computing the silhouette score for the clusters identified with DBSCAN in order to fine tuned these two parameters.